**3D-Cube Orientation**

Class data structure and operations relate
to a Rubik's Cube as in this orientation:

![alt text](rubiks_web.png)
![alt text](rubiks-cube.jpg)

In [50]:
import math
import numpy as np

# COLORS/SIDES:
WHITE, W  = 1, 1
ORANGE, O = 2, 2
GREEN, G  = 3, 3
RED, R    = 4, 4
BLUE, B   = 5, 5
YELLOW, Y = 6, 6

# COLORS/SIDES in order
SIDES = [WHITE, ORANGE, GREEN, RED, BLUE, YELLOW]

# Print convenience maps/fcns
color_letr_map = { 0:0, WHITE:'W', ORANGE:'O', GREEN:'G', RED:'R', BLUE:'B', YELLOW:'Y' }
color_name_map = { 0:'N/A', WHITE:'WHITE', ORANGE:'ORANGE', GREEN:'GREEN', RED:'RED', BLUE:'BLUE', YELLOW:'YELLOW' }

def color_letr(fc): return color_letr_map[fc]
def color_name(fc): return color_name_map[fc]
def tri_color_name(tc): return (f"({color_letr(tc[0])},{color_letr(tc[1])},{color_letr(tc[2])})")

In [51]:
class Facelet:
#{
    def __init__(self, color, index, position):
        self.color = color
        self.index = index
        self.position = position

    def __repr__(self):
        return f"{color_name(self.color)}-{self.index}"

    def __eq__(self, other):
        return (self.color == other.color) and (self.position == other.position)

    def matches(self, color, index):
        return (self.color == color) and (self.index == index)
    
    def copy(self):
        return Facelet(color=self.color, index=self.index, position=self.position)

    def name(self):
        return f"{color_name(self.color)}-{self.index} : {self.position}"

    def state(self):
        return np.concatenate(([self.color], [self.index], self.position))

    def dot(self, other):
        return self.position.dot(other.position)

    def isat(self, position):
        return sum(self.position == position) == 3
    
    # NO check for isonside(...)
    def apply_rotation(self, R):
        self.position = R.dot(self.position)
#}

class Spiderweb:
#{
    def __init__(self, flet_a, flet_b):
        self.flet_a = flet_a
        self.flet_b = flet_b
    
    def __repr__(self):
        return self.flet_a.__repr__() + " to " + self.flet_b.__repr__()
    
    def distance(self):
        return np.linalg.norm(self.flet_b.position - self.flet_a.position) #- self.d_factor
#}

In [52]:
class SpiCube:
#{
    # Solved facelet position vectors, ordered left-to-right top-to-bottom per side
    SOLVED_POS = { WHITE:   np.array([[-2,-2,3],[-2,0,3],[-2,2,3],[0,-2,3],[0,0,3],[0,2,3],[2,-2,3],[2,0,3],[2,2,3]]),
                   ORANGE:  np.array([[-2,-3,2],[0,-3,2],[2,-3,2],[-2,-3,0],[0,-3,0],[2,-3,0],[-2,-3,-2],[0,-3,-2],[2,-3,-2]]),
                   GREEN:   np.array([[3,-2,2],[3,0,2],[3,2,2],[3,-2,0],[3,0,0],[3,2,0],[3,-2,-2],[3,0,-2],[3,2,-2]]),
                   RED:     np.array([[2,3,2],[0,3,2],[-2,3,2],[2,3,0],[0,3,0],[-2,3,0],[2,3,-2],[0,3,-2],[-2,3,-2]]),
                   BLUE:    np.array([[-3,2,2],[-3,0,2],[-3,-2,2],[-3,2,0],[-3,0,0],[-3,-2,0],[-3,2,-2],[-3,0,-2],[-3,-2,-2]]),
                   YELLOW:  np.array([[2,-2,-3],[2,0,-3],[2,2,-3],[0,-2,-3],[0,0,-3],[0,2,-3],[-2,-2,-3],[-2,0,-3],[-2,2,-3]]) }


    # Fixed cube facelet centers
    CENTERS = { WHITE:  Facelet(color=WHITE,  index=4, position=np.array([ 0,  0,  3])),
                ORANGE: Facelet(color=ORANGE, index=4, position=np.array([ 0, -3,  0])),
                GREEN:  Facelet(color=GREEN,  index=4, position=np.array([ 3,  0,  0])),
                RED:    Facelet(color=RED,    index=4, position=np.array([ 0,  3,  0])),
                BLUE:   Facelet(color=BLUE,   index=4, position=np.array([-3,  0,  0])),
                YELLOW: Facelet(color=YELLOW, index=4, position=np.array([ 0,  0, -3])) }
    
    # The 18 possible moves from any cube state
    MOVES = [(sd,ang) for sd in SIDES for ang in [90,-90,180]]
    
    # Access through static methods below
    _rotation_matrices = {}
    
    @staticmethod
    def rotation(side_angle_tuple):
    #{
        if not SpiCube._rotation_matrices:
        #{
            # Rotation matrix mappings
            for rad, deg in zip([math.pi/2, -math.pi/2, math.pi], [90, -90, 180]):
            #{
                c = round(math.cos(rad))
                s = round(math.sin(rad))

                Rx = np.array([[1,0,0],[0,c,-s],[0,s,c]])
                Ry = np.array([[c,0,s],[0,1,0],[-s,0,c]])
                Rz = np.array([[c,-s,0],[s,c,0],[0,0,1]])

                SpiCube._rotation_matrices[(GREEN,deg)]  = Rx
                SpiCube._rotation_matrices[(BLUE,deg)]   = Rx
                SpiCube._rotation_matrices[(RED,deg)]    = Ry
                SpiCube._rotation_matrices[(ORANGE,deg)] = Ry
                SpiCube._rotation_matrices[(WHITE,deg)]  = Rz
                SpiCube._rotation_matrices[(YELLOW,deg)] = Rz
            #}
        #}
        
        return SpiCube._rotation_matrices[side_angle_tuple]
    #}
    
    # Dist scale factor
    _DFACTOR = 1.0
    
    # Begin SpiCube implementation
    def __init__(self, copycube=None):
    #{
        # Construct facelets
        if copycube is not None: self.facelets = [copylet.copy() for copylet in copycube.facelets]
        else: self.facelets = [Facelet(side, ind, pos) for side in SIDES for ind, pos in 
                               zip(range(9), SpiCube.SOLVED_POS[side]) if (ind != 4)]

        # Connect webs
        self.spiderwebs = []
        for flet in self.facelets:
        #{
            # Connect corner and edge facelets to fixed center facelet
            self.spiderwebs.append(Spiderweb(flet, SpiCube.CENTERS[flet.color]))

            # Connect corner facelets to their adjacent edge facelets
            if flet.index == 0:
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 1)))
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 3)))
            if flet.index == 2:
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 1)))
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 5)))
            if flet.index == 6:
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 3)))
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 7)))
            if flet.index == 8:
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 5)))
                self.spiderwebs.append(Spiderweb(flet, self.get_facelet(flet.color, 7)))   
        #}
        
        # Lazy init upon first cube constructed
        if SpiCube._DFACTOR == 1.0: SpiCube._DFACTOR = self.distance()
    #}
    
    def isinlayer(self, flet, side):
        return flet.dot(SpiCube.CENTERS[side]) > 0
    
    def isonside(self, flet, side):
        return flet.dot(SpiCube.CENTERS[side]) >= 9
    
    def get_facelets(self, side, layer=True):
        if layer: return [flet for flet in self.facelets if self.isinlayer(flet, side)]
        else:     return [flet for flet in self.facelets if self.isonside(flet, side)]

    def get_facelet(self, color, index):
        return next(flet for flet in self.facelets if flet.matches(color, index))
    
    def get_colorat(self, side, position):
    #{
        # Facelet will default to None if position is one of the 6 fixed centers 
        facelet = next((flet for flet in self.facelets if flet.isat(position)), None)
        return facelet.color if facelet is not None else side
    #}
    
#     def numb_solved(self):
#         return sum([Cube3D.clet_is_solved(clet) for clet in self.cubelets])
    
    def is_solved(self):
        return self.distance() == 1.0

    def state(self):
        return np.concatenate([flet.state() for flet in self.facelets])
    
    def reset(self):
        for flet in self.facelets: flet.position = SpiCube.SOLVED_POS[flet.color][flet.index]
            
    def distance(self):
        return sum([web.distance() for web in self.spiderwebs]) / SpiCube._DFACTOR
    
    def scramble(self, sz=64):
    #{
        self.reset() # Start from solved/reset state
        angle = np.random.choice([90, -90, 180], size=sz)
        sides = np.random.randint(low=1, high=7, size=sz)
        for move in zip(sides, angle): self.rotate(move)
        return self
    #}
    
    def rotate(self, move):
        for flet in self.get_facelets(side=move[0]): flet.apply_rotation(SpiCube.rotation(move))
        return self
#}

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import matplotlib.colors as mpc

# Plotting color map
color_plot_map = { WHITE: mcd.CSS4_COLORS['ivory'],
                   ORANGE: mcd.XKCD_COLORS['xkcd:orange'].upper(), 
                   GREEN: mcd.CSS4_COLORS['green'],
                   RED: mcd.XKCD_COLORS['xkcd:crimson'].upper(),
                   BLUE: mcd.XKCD_COLORS['xkcd:blue'].upper(),
                   YELLOW: mcd.XKCD_COLORS['xkcd:goldenrod'].upper() }

def color(fc): return color_plot_map[fc]

In [5]:
class SpiCubeView:
#{
    OFFSET     = 130
    CENTERS    = np.array([[0,0,3],[0,-3,0],[3,0,0],[0,3,0],[-3,0,0],[0,0,-3]])
    ANCHOR_POS = { WHITE:(40,70), ORANGE:(10,40), GREEN:(40,40), RED:(70,40), BLUE:(100,40), YELLOW:(40,10) }
    
    
    def __init__(self, scube):
        self.spider_cube = scube
        self.patch_sequence = [[]]
        self.caption_sequence = [[]]
    
    def reset_snapshots(self):
        self.patch_sequence = []
        self.caption_sequence = []
    
    def iscenter(self, position):
        return next((True for cent in self.CENTERS if sum(cent == position) == 3), False)
    
    def get_plot_rects(self, faces, anchor=(0,0), mask=None):
    #{    
        rects = []
        for r in range(3):
            for c in range(3):
                
                x = anchor[0] + (c*10)
                y = anchor[1] + 20  - (r*10)
                clr = color(faces[r,c])
                
                if mask is not None and (mask[r,c] == 'gray'):
                    g = np.mean(mpc.to_rgb(clr)) * 0.667
                    clr = mpc.to_hex((g,g,g))
                  
                rects.append(plt.Rectangle((x, y), 10, 10, fc=clr))

        return rects
    #}
    
    def get_gridlines(self, seqnumb=0):
    #{
        lines = []            
        for anchor in [(40,70), (10,40), (40,40), (70,40), (100,40), (40,10)]:
        #{
            xoff = self.OFFSET * seqnumb
        
            for r in range(4):
                y = anchor[1] + (r*10)
                lines.append(plt.Line2D((anchor[0] + xoff, anchor[0] + xoff + 30), (y, y), lw=1, color='k'))

            for c in range(4):
                x = anchor[0] + xoff + (c*10)
                lines.append(plt.Line2D((x, x), (anchor[1], anchor[1] + 30), lw=1, color='k'))
        #}
        
        return lines
    #}
    
    def create_patches(self, cubelet=None, seqnumb=0):
    #{
        rects = []
        mask = None
        
        for side in SIDES:
            anchor = (self.ANCHOR_POS[side][0] + (self.OFFSET * seqnumb), self.ANCHOR_POS[side][1])
            faces = [self.spider_cube.get_colorat(side, pos) for pos in SpiCube.SOLVED_POS[side]]
            rects.extend(self.get_plot_rects(np.reshape(faces, (3,3)), anchor, mask))
        
        return rects
    #}
    
    # Displays a single cube projection
    def draw_projection(self, cubelet=None):
    #{
        fig = plt.figure(figsize=[4, 3])
        ax = fig.add_axes([0, 0, 1, 1])
        
        rects = self.create_patches(cubelet)
        for r in rects: ax.add_patch(r)
        for ln in self.get_gridlines(): ax.add_line(ln)
 
        ax.axis('scaled')
        ax.axis('off')
        plt.show()
    #}
    
    def push_snapshot(self, cubelet=None, caption=""):
    #{
        # Adjust to 5 cube images per row
        top = len(self.patch_sequence)-1
        if len(self.patch_sequence[top]) > 4:
            self.caption_sequence.append([])
            self.patch_sequence.append([])
            top = len(self.patch_sequence)-1

        seqnumb = len(self.patch_sequence[top])
        rects = self.create_patches(cubelet, seqnumb)
        self.patch_sequence[top].append(rects)
        self.caption_sequence[top].append(caption)
    #}
    
    # Displays a move sequence
    def draw_snapshops(self):
    #{
        for row in range(len(self.patch_sequence)):
        #{
            nmoves = len(self.patch_sequence[row])
            width = (4 * nmoves) + (nmoves - 1)
            fig = plt.figure(figsize=[width, 3])
            ax = fig.add_axes([0, 0, 1, 1])

            for seqnumb in range(len(self.patch_sequence[row])):
            #{
                rects = self.patch_sequence[row][seqnumb]
                caption = self.caption_sequence[row][seqnumb]

                for r in rects: ax.add_patch(r)
                for ln in self.get_gridlines(seqnumb): ax.add_line(ln)
                plt.text(20 +(self.OFFSET * seqnumb), 110, caption)
            #}

            ax.axis('scaled')
            ax.axis('off')
        #}
        
        plt.show()
    #}
#}

In [6]:
class UCTNode:
#{
    UCTFACTOR = math.sqrt(2.0)

    def __init__(self, distance=2.25, action=None, parent=None):
    #{
        # Action that produced this node/state
        self.action = action
        
        self.numb_visits = 0
        self.child_nodes = []
        self.parent_node = parent

        # Value ends up in set (0, 1.25] 
        self.value = 2.25 - distance
        self.avg_value = self.value
    #}
    
    def __repr__(self):
        return (f"Action:  {self.action}\n"
                f"Visits:  {self.numb_visits}\n"
                f"Value:   {self.value}\n"
                f"Avg Val: {self.avg_value})")
    
    def set_root(self):
        self.parent_node = None
        return self
    
    def ucbonus(self):
    #{
        # Calc uncertainty bonus
        exploit = (1 + self.numb_visits)
        explore = np.log(self.parent_node.numb_visits)
        return UCTNode.UCTFACTOR * math.sqrt(explore/exploit)
    #}
    
    def select_uct_child(self):
        return max(self.child_nodes, key=lambda cld: cld.avg_value + cld.ucbonus())
    
    def select_fav_child(self):
        return max(self.child_nodes, key=lambda cld: cld.numb_visits)
    
    def add_child(self, action):
    #{
        child = UCTNode(action=action, parent=self)
        self.child_nodes.append(child)
        return child
    #}
    
    def update(self, distance):
    #{
        self.numb_visits += 1
        self.value += (2.25 - distance)
        self.avg_value = self.value/self.numb_visits
    #}
#}

In [41]:
import keras, random, time
from keras.models import Model
from keras.layers import Dense, Input

class SpiCubeSolver:
#{
    def __init__(self):
    #{
        # Hyper-params
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.05
    #}

    def create_spicube_model(self, summarize=True):
    #{  
        # Prime input, connect layers, get ref to output
        X_input  = Input(shape=(240,))
        X        = Dense(units=240, activation='relu')(X_input)
        X        = Dense(units=240, activation='relu')(X)
        Q_output = Dense(units=18, activation='linear')(X)
                
        # Construct and compile the full-cube model
        cube_model = Model(inputs=X_input, outputs=Q_output)
        cube_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
        
        if summarize: cube_model.summary()        
        return cube_model
    #}
    
    def nn_state(self, cube):
        return cube.state().reshape((1,240))
    
    def step_cube(self, action, cube):
    #{
        # Update cube state
        cube.rotate(SpiCube.MOVES[action])
        
        # Calc reward and if done
        reward = -cube.distance()
        solved = (reward == 0)
        if not solved: reward -= 10
        
        return reward, solved
    #}
    
    # Plays max of 128 moves/rotations trying to solve the cube
    def solve_cube(self, model, cube, nmoves=128, training=False):
    #{
        start = time.time()
        
        gamememory = []
        rootnode = UCTNode(cube.distance())
        
        for mvnum in range(nmoves):
        #{
            # Initial state
            state  = self.nn_state(cube)
            
            # Determine and apply the best next move
            vnode = self.uc_tree_search(model, cube, rootnode)
            solved = cube.rotate(SpiCube.MOVES[vnode.action]).is_solved()

            # Store training information
            gamememory.append((state, vnode.action, vnode.avg_value, self.nn_state(cube), solved))
            
            if solved: break
            else: rootnode = vnode.set_root()
        #}
        
        print(f"{nmoves} cube solution attempt: {time.time() - start} sec")
        return gamememory
    #}
    
    def train_cube_model(self, model, playmemory, batch_size=128):
    #{
        if len(playmemory) < batch_size: batch_size = len(playmemory)
        self.epsilon = max(self.epsilon_min, self.epsilon_decay*self.epsilon)

        in_states = np.zeros((batch_size, 240))
        y_truish  = np.zeros((batch_size, len(SpiCube.MOVES)))
        Qs_solved = np.zeros((batch_size, len(SpiCube.MOVES)))
        minibatch = random.sample(playmemory, batch_size)
        
        for m in range(batch_size):
        #{
            # Unpack play memory
            state, action, reward, state_p, solved = minibatch[m]
        
            # Batch of input states 
            in_states[m] = state
            
            # Output target batch (r + discounted Rt+1)
            Qs_p = Qs_solved if solved else model.predict(state_p, batch_size=1)
            y_truish[m] = model.predict(state, batch_size=1)
            y_truish[m, action] = reward + (self.gamma * np.amax(Qs_p))            
        #}
                
        return model.fit(in_states, y_truish, batch_size=batch_size, epochs=1, verbose=0)
    #}
    
    def learn_cube(self, cube_model, episodes=10000):
    #{
        learning_cube = SpiCube()
        learnmemory, history = [], []
        
        self.epsilon = 1.0
        for j in range(episodes+1):
        #{
            # Reset env and train
            learning_cube.scramble()                
            learnmemory.extend(self.solve_cube(cube_model, learning_cube, training=True))
            history.append(self.train_cube_model(cube_model, learnmemory))
            if (j % 2000) == 0: print(f"  Training SpiCube: episode {j} of {episodes}")
        #}
        
        return history
    #}
        
    def uc_tree_search(self, model, spicube, rootnode, iterations=1000):
    #{
        start = time.time()
        proto_avg = {'ctor': 0,
                     'select': 0,
                     'expand': 0,
                     'backup': 0}
        
        # UCT loop (w/no MC rollout)
        for i in range(iterations):
        #{
            initial = time.time()
            
            node = rootnode
            cube = SpiCube(spicube)
            
            ctor = time.time()

            # Select (down to leaf)
            while node.child_nodes:
                node = node.select_uct_child()
                cube.rotate(SpiCube.MOVES[node.action])
                
            select = time.time()

            # Expand fully and evaluate via NN
            for action in range(len(SpiCube.MOVES)): node.add_child(action)
            action = np.argmax(model.predict(self.nn_state(cube), batch_size=1))
            dist = cube.rotate(SpiCube.MOVES[action]).distance()
            node = node.child_nodes[action]
            
            expand = time.time()

            # Backup
            while node:
                node.update(dist)
                node = node.parent_node
                
            backup = time.time()
            
            proto_avg['ctor'] += (ctor - initial)
            proto_avg['select'] += (select - ctor)
            proto_avg['expand'] += (expand - select)
            proto_avg['backup'] += (backup - expand)
        #}
        
        vnode = rootnode.select_fav_child()
        
        duration = time.time() - start
        a = proto_avg['ctor']  / duration
        b = proto_avg['select'] / duration
        c = proto_avg['expand'] / duration
        d = proto_avg['backup'] / duration
        
        print(f"  {iterations} UCT search iterations: {time.time() - start} sec")
        print(f"    ctor: {a}, select: {b}, expand: {c}, backup: {d}")
              
        return vnode
    #}
#}

In [42]:
class SpiCubeTest:
#{
    @staticmethod
    def moving_average(a, n=10):
    #{
        ret = np.cumsum(a, dtype=float)
        ret[n:] = ret[n:] - ret[:-n]
        return ret[n - 1:] / n
    #}
    
    @staticmethod
    def extract_acc_loss(history_list):
    #{
        accuracy,loss = [],[]
        for h in history_list:
            accuracy.extend(h.history['acc'])
            loss.extend(h.history['loss'])

        accuracy = SpiCubeTest.moving_average(accuracy, n=10)
        loss = SpiCubeTest.moving_average(loss, n=10)

        return accuracy, loss
    #}
    
    def run_cube_test(self, model):
    #{
        spicube = SpiCube()        
        spicube.scramble()
        spicube_b = SpiCube(spicube)
        spicube_c = SpiCube(spicube)
        
        solver = SpiCubeSolver()
        game = solver.solve_cube(model, spicube)
        print("Running Rubik\'s solve_cube test...")

        for state, action, reward, state_p, solved in game:
        #{
            # Apply the action to the actual cube
            side, angle = SpiCube.MOVES[action]
            spicube_b.rotate(move=(side, angle))
        
            transition  = "SOLVED-to-" if spicube_b.is_solved() else "UNSOLVED-to-"
            transition += "SOLVED" if spicube_b.is_solved() else "UNSOLVED"

            print((f"  {color_name(side)}({angle}) : {transition} : "
                   f"reward {reward} : solved {solved}"))
        #}

        print("\n")
        mv_number = 1
        cube_view = SpiCubeView(spicube_c)
        cube_view.push_snapshot(caption=f"Initial cube state...")
        for _, action, _, _, _ in game:
        #{        
            # Apply the action to the actual cube
            side, angle = SpiCube.MOVES[action]
            spicube_c.rotate(move=(side, angle))
            
            caption  = f"Move {mv_number}: {color_name(side)}({angle})"
            if spicube_c.is_solved(): caption += " : SOLVED!!"
            mv_number += 1
            
            cube_view.push_snapshot(caption=caption)
            if spicube_c.is_solved(): break
        #}

        cube_view.draw_snapshops()
    #}
#}

In [43]:
solver = SpiCubeSolver()
model = solver.create_spicube_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 240)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_23 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_24 (Dense)             (None, 18)                4338      
Total params: 120,018
Trainable params: 120,018
Non-trainable params: 0
_________________________________________________________________


In [44]:
history = solver.learn_cube(model)

  1000 UCT search iterations: 2.2158751487731934 sec
    ctor: 0.22329502385667382, select: 0.18832123149823354, expand: 0.5842272121557406, backup: 0.003502888139663623
  1000 UCT search iterations: 2.018656015396118 sec
    ctor: 0.19680851754885437, select: 0.2110414393831755, expand: 0.5876819109446737, backup: 0.0037852334439884305
  1000 UCT search iterations: 2.1195099353790283 sec
    ctor: 0.2355660004132798, select: 0.20097232116452665, expand: 0.5592057063659376, backup: 0.0036143422264982546
  1000 UCT search iterations: 2.102207899093628 sec
    ctor: 0.23217708576345536, select: 0.20258839255204011, expand: 0.5609118768618964, backup: 0.003651803553632767
  1000 UCT search iterations: 2.015406847000122 sec
    ctor: 0.19823283929285654, select: 0.21101518580905432, expand: 0.5862600776674359, backup: 0.0038037571713788153
  1000 UCT search iterations: 2.1288089752197266 sec
    ctor: 0.24047269619069123, select: 0.20023868657871818, expand: 0.5550463082425715, backup: 0.0

KeyboardInterrupt: 

In [ ]:
accuracy, loss = SpiCubeTest.extract_acc_loss(history)

# Set up a wider notebook plot display size
#print(matplotlib.rcParams['figure.figsize'])
matplotlib.rcParams['figure.figsize'] = (20, 4)

# Plot with respect to accuracy
plt.figure(1)
plt.plot(accuracy)
plt.title(f'Rubik\'s Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Episode')

# Plot with respect to loss
plt.figure(2)
plt.plot(loss)
plt.title(f'Rubik\'s Model Loss')
plt.ylabel('Loss')
plt.xlabel('Episode')

In [ ]:
SpiCubeTest().run_cube_test(model)

In [ ]:
# Explore average distance value at x in [0,32] scramble-moves out

spicube = SpiCube()
matplotlib.rcParams['figure.figsize'] = (20, 4)

print(spicube.__dict__())

dist =[np.mean([spicube.scramble(sz=i).distance() for tests in range(100)]) for i in range(33)]
dist_exp = np.power(30, dist/np.mean(dist))
dist_inv_a = [1.0/d for d in dist]
dist_inv_b = [1.0/dx for dx in dist_exp]
print(2.25 - np.max(dist), 2.25 - np.min(dist))

# Plot
plt.figure(1)
plt.plot(np.subtract(2.25, dist))
plt.title(f'Rubik\'s Cube Values')
plt.ylabel('Avg Value')
plt.xlabel('Number of Shuffle Turns')

# # Plot (scaled)
# plt.figure(2)
# plt.plot(dist_exp)
# plt.title(f'Rubik\'s Web Exp Distances')
# plt.ylabel('Avg Spider Exp Distance')
# plt.xlabel('Number of Shuffle Turns')

# # Plot (scaled)
# plt.figure(3)
# plt.plot(dist_inv_a)
# plt.plot(dist_inv_b)
# plt.title(f'Rubik\'s Web Inv Distances')
# plt.ylabel('Avg Spider Log Distance')
# plt.xlabel('Number of Shuffle Turns')

In [ ]:
for i in range(20):
#{
    dist_list = []
    for move in SpiCube.MOVES:
    #{
        cube_t = SpiCube(cube)
        cube_t.rotate(move)
        dist_list.append(cube_t.distance())
    #     print(f"  {color_name(side)}({angle}) : {cube_t.distance()}")
    #}  

    index = np.argmin(dist_list)
    side, angle = SpiCube.MOVES[index]
    cube.rotate(move=(side, angle))
    print(f"  {color_name(side)}({angle}) : {dist_list[index]}")
#}

SpiCubeView(cube).draw_projection()

In [ ]:
def posisin(pos_t, dd_list):
#{
    for dd in dd_list:
        if pos_t in dd: return True
    return False
#}

def pathisin(path_t, pt_list):
#{
    for pt in pt_list:
        if path_t == pt: return True
    return False
#}

def custom_scramble(spicube, sz=3):
#{
    paths = []

    spicube.reset() # Start from solved/reset state
    angle = np.random.choice([90, -90, 180], size=sz)
    sides = np.random.randint(low=1, high=7, size=sz)
    for move in zip(sides, angle):
        spicube.rotate(move)
        paths.append(move)
        
    return tuple(paths)
#}

spicube = SpiCube()
custom_scramble(spicube)



In [ ]:

MOVES = [(sd,ang) for sd in SIDES for ang in [90,-90,180]]

def tfucntion(action):
#     side, angle = action
    print(*action)
    
for move in MOVES: tfucntion(move)

In [ ]:

distances = []
spicube = SpiCube()

for sz in range(4):
#{
    distances.append({})
    dd = distances[sz]
    
    for test in range(100000):
    #{
        path_t = custom_scramble(spicube, sz=sz)
        pos_t = tuple(spicube.facelets[0].position)
        
        if not posisin(pos_t, distances[0:sz]):
            if pos_t not in dd: dd[pos_t] = []
            if not pathisin(path_t, dd[pos_t]):
                dd[pos_t].append(path_t)
    #}
#}

# spicube = SpiCube()
# spicube.scramble(sz=2)

# scube_view = SpiCubeView(spicube)
# scube_view.draw_projection()

In [ ]:
for i in range(len(distances)):
    index = 1
    print("Distance", i, "positions")
    for k,v in sorted(distances[i].items()):
        print(index, ":", k, ": unique paths:", len(v))
        index += 1

In [ ]:
spider_cube = SpiCube()

print("WHITE", spider_cube.get_facelets(WHITE, layer=False))
print("ORANGE", spider_cube.get_facelets(ORANGE, layer=False))
print("GREEN", spider_cube.get_facelets(GREEN, layer=False))
print("RED", spider_cube.get_facelets(RED, layer=False))
print("BLUE", spider_cube.get_facelets(BLUE, layer=False))
print("YELLOW", spider_cube.get_facelets(YELLOW, layer=False))

cube_view = SpiCubeView(spider_cube)
cube_view.draw_projection()

spider_cube.scramble()
print("WHITE", spider_cube.get_facelets(WHITE, layer=False))
print("ORANGE", spider_cube.get_facelets(ORANGE, layer=False))
print("GREEN", spider_cube.get_facelets(GREEN, layer=False))
print("RED", spider_cube.get_facelets(RED, layer=False))
print("BLUE", spider_cube.get_facelets(BLUE, layer=False))
print("YELLOW", spider_cube.get_facelets(YELLOW, layer=False))

cube_view.draw_projection()

In [ ]:
# for i, j in zip(range(1,6), ['a','b','c','d','e']):
#     print(i,j)

a = np.array([-2,-2,3])
b = np.array([-3,-2,-2])
np.linalg.norm(b - a)

In [ ]:
for pos in SpiCube.SOLVED_POS[WHITE]:
    print(np.linalg.norm(pos) > 3.0)

In [ ]:
# side = 1
# code_line = ""
# for i in range(len(spider_cube.facelets)):
    
#     if (i % 8) == 0:
#         if code_line != "":
#             code_line += f"]),"
#             print(code_line)
#             code_line = ""
    
#         code_line += color_name(side)
#         code_line += f":  np.array(["
    
#     code_line += f"[{spider_cube.facelets[i].position[0]},{spider_cube.facelets[i].position[1]},{spider_cube.facelets[i].position[2]}],"
    
#     if spider_cube.facelets[i].index == 4:
#         code_line += f"[{SpiCube.CENTERS[side].position[0]},{SpiCube.CENTERS[side].position[1]},{SpiCube.CENTERS[side].position[2]}],"
#         side += 1

# code_line += f"])"
# print(code_line)
# code_line = ""

# spider_cube.scramble()

In [ ]:
#     # Begin SpiCube implementation
#     def __init__(self, copycube=None):
#     #{
#         if copycube is None:
#         #{
#             self.facelets = []
            
#              # WHITE side
#             self.facelets.append(Facelet(color=WHITE, index=1, position=np.array([-2, -2,  3])))
#             self.facelets.append(Facelet(color=WHITE, index=2, position=np.array([-2,  0,  3])))
#             self.facelets.append(Facelet(color=WHITE, index=3, position=np.array([-2,  2,  3])))
#             self.facelets.append(Facelet(color=WHITE, index=4, position=np.array([ 0, -2,  3])))
#             self.facelets.append(Facelet(color=WHITE, index=6, position=np.array([ 0,  2,  3])))
#             self.facelets.append(Facelet(color=WHITE, index=7, position=np.array([ 2, -2,  3])))
#             self.facelets.append(Facelet(color=WHITE, index=8, position=np.array([ 2,  0,  3])))
#             self.facelets.append(Facelet(color=WHITE, index=9, position=np.array([ 2,  2,  3])))

#             # ORANGE side
#             self.facelets.append(Facelet(color=ORANGE, index=1, position=np.array([-2, -3,  2])))
#             self.facelets.append(Facelet(color=ORANGE, index=2, position=np.array([ 0, -3,  2])))
#             self.facelets.append(Facelet(color=ORANGE, index=3, position=np.array([ 2, -3,  2])))
#             self.facelets.append(Facelet(color=ORANGE, index=4, position=np.array([-2, -3,  0])))
#             self.facelets.append(Facelet(color=ORANGE, index=6, position=np.array([ 2, -3,  0])))
#             self.facelets.append(Facelet(color=ORANGE, index=7, position=np.array([-2, -3, -2])))
#             self.facelets.append(Facelet(color=ORANGE, index=8, position=np.array([ 0, -3, -2])))
#             self.facelets.append(Facelet(color=ORANGE, index=9, position=np.array([ 2, -3, -2])))

#             # GREEN side
#             self.facelets.append(Facelet(color=GREEN, index=1, position=np.array([ 3, -2,  2])))
#             self.facelets.append(Facelet(color=GREEN, index=2, position=np.array([ 3,  0,  2])))
#             self.facelets.append(Facelet(color=GREEN, index=3, position=np.array([ 3,  2,  2])))
#             self.facelets.append(Facelet(color=GREEN, index=4, position=np.array([ 3, -2,  0])))
#             self.facelets.append(Facelet(color=GREEN, index=6, position=np.array([ 3,  2,  0])))
#             self.facelets.append(Facelet(color=GREEN, index=7, position=np.array([ 3, -2, -2])))
#             self.facelets.append(Facelet(color=GREEN, index=8, position=np.array([ 3,  0, -2])))
#             self.facelets.append(Facelet(color=GREEN, index=9, position=np.array([ 3,  2, -2])))

#             # RED side
#             self.facelets.append(Facelet(color=RED, index=1, position=np.array([ 2,  3,  2])))
#             self.facelets.append(Facelet(color=RED, index=2, position=np.array([ 0,  3,  2])))
#             self.facelets.append(Facelet(color=RED, index=3, position=np.array([-2,  3,  2])))
#             self.facelets.append(Facelet(color=RED, index=4, position=np.array([ 2,  3,  0])))
#             self.facelets.append(Facelet(color=RED, index=6, position=np.array([-2,  3,  0])))
#             self.facelets.append(Facelet(color=RED, index=7, position=np.array([ 2,  3, -2])))
#             self.facelets.append(Facelet(color=RED, index=8, position=np.array([ 0,  3, -2])))
#             self.facelets.append(Facelet(color=RED, index=9, position=np.array([-2,  3, -2])))

#             # BLUE side
#             self.facelets.append(Facelet(color=BLUE, index=1, position=np.array([-3,  2,  2])))
#             self.facelets.append(Facelet(color=BLUE, index=2, position=np.array([-3,  0,  2])))
#             self.facelets.append(Facelet(color=BLUE, index=3, position=np.array([-3, -2,  2])))
#             self.facelets.append(Facelet(color=BLUE, index=4, position=np.array([-3,  2,  0])))
#             self.facelets.append(Facelet(color=BLUE, index=6, position=np.array([-3, -2,  0])))
#             self.facelets.append(Facelet(color=BLUE, index=7, position=np.array([-3,  2, -2])))
#             self.facelets.append(Facelet(color=BLUE, index=8, position=np.array([-3,  0, -2])))
#             self.facelets.append(Facelet(color=BLUE, index=9, position=np.array([-3, -2, -2])))

#             # YELLOW side
#             self.facelets.append(Facelet(color=YELLOW, index=1, position=np.array([ 2, -2, -3])))
#             self.facelets.append(Facelet(color=YELLOW, index=2, position=np.array([ 2,  0, -3])))
#             self.facelets.append(Facelet(color=YELLOW, index=3, position=np.array([ 2,  2, -3])))
#             self.facelets.append(Facelet(color=YELLOW, index=4, position=np.array([ 0, -2, -3])))
#             self.facelets.append(Facelet(color=YELLOW, index=6, position=np.array([ 0,  2, -3])))
#             self.facelets.append(Facelet(color=YELLOW, index=7, position=np.array([-2, -2, -3])))
#             self.facelets.append(Facelet(color=YELLOW, index=8, position=np.array([-2,  0, -3])))
#             self.facelets.append(Facelet(color=YELLOW, index=9, position=np.array([-2,  2, -3])))
#         #}
#         else: self.facelets = [copylet.copy() for copylet in copycube.facelets]
#     #}
